# 12) Clustering Overview & Config
Bu bölüm, Part 11'den sonraki **Clustering** adımlarını içerir. LSH/MinHash tarafından üretilmiş **imza tablolarını** (title + s1..sK) kullanarak **K-Means** ve **DBSCAN** ile kümeleme yapar; per-type ve combined (multi-view) akışları destekler.

**Çıktılar** `out/part12_17_clustering/` dizinine yazılır:
- `{type}_kmeans_k{K}.csv`, `{type}_kmeans_pca.png`
- `{type}_dbscan_eps{E}_min{M}.csv`, `{type}_dbscan_pca.png`
- `combined_*` çıktıları (çoklu görünüm)

In [ ]:
import os, numpy as np, pandas as pd
from pathlib import Path
from typing import List, Tuple, Dict

from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

RANDOM_STATE = 0
K_CANDIDATES = [5, 8, 10, 12, 15]
SCALE_FEATURES = True

# DBSCAN süpürme grid'i (gerektiğinde değiştir)
DBSCAN_EPS_GRID = [0.2, 0.3, 0.4, 0.5, 0.6]
DBSCAN_MIN_SAMPLES = [3, 5, 8]

OUT_DIR = Path('out/part12_17_clustering')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# İmza dosyaları — mevcut değilse atlanır. Arkadaşınız 17 parçayı tamamladığında # buraya yeni satırlar ekleyebilirsiniz.
SIGNATURE_FILES = [
    ('title',       'outputs/signatures_title.csv'),
    ('description', 'outputs/signatures_description.csv'),
    ('cast',        'outputs/signatures_cast.csv'),
    ('director',    'outputs/signatures_director.csv'),
    ('country',     'outputs/signatures_country.csv'),
    ('year',        'outputs/signatures_year.csv'),
    ('rating',      'outputs/signatures_rating.csv'),
    ('type',        'outputs/signatures_type.csv'),
    # ('language',   'outputs/signatures_language.csv'),
    # ('production', 'outputs/signatures_production.csv'),
]


# 13) Yardımcı Fonksiyonlar (IO, Ölçekleme, Görselleştirme, Değerlendirme)

In [ ]:
def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def load_signatures(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    if 'title' not in df.columns:
        raise ValueError(f"'title' column missing in {csv_path}")
    sig_cols = [c for c in df.columns if c.startswith('s')]
    if not sig_cols:
        raise ValueError(f"No signature columns s1..sK in {csv_path}")
    return df[['title'] + sig_cols]

def get_feature_matrix(df: pd.DataFrame) -> Tuple[np.ndarray, List[str]]:
    sig_cols = [c for c in df.columns if c.startswith('s')]
    X = df[sig_cols].values
    if SCALE_FEATURES:
        X = StandardScaler(with_mean=False).fit_transform(X)
    return X, sig_cols

def pca_plot(X: np.ndarray, labels: np.ndarray, title: str, out_path: Path):
    pca = PCA(n_components=2, random_state=RANDOM_STATE)
    reduced = pca.fit_transform(X)
    plt.figure(figsize=(5,4))
    plt.scatter(reduced[:,0], reduced[:,1], c=labels, s=10)
    plt.title(title)
    plt.xlabel('PCA1'); plt.ylabel('PCA2')
    plt.tight_layout(); plt.savefig(out_path, dpi=150); plt.close()

def preview_clusters(titles: pd.Series, labels: np.ndarray, per_cluster:int=3):
    df_tmp = pd.DataFrame({'title': titles, 'cluster': labels})
    for c in sorted(df_tmp['cluster'].unique()):
        ex = df_tmp.loc[df_tmp.cluster==c, 'title'].head(per_cluster).to_list()
        print(f"  Cluster {c}: {ex}")


# 14) K-Means (Per-Type)
Her imza tipi için en iyi `k` değerini (silhouette) seçip K-Means uygular; sonuçları kaydeder.

In [ ]:
km_summary = []
for name, path in SIGNATURE_FILES:
    p = Path(path)
    if not p.exists():
        print(f"[SKIP] {name}: {path} bulunamadı.")
        continue

    df = load_signatures(str(p))
    X, sig_cols = get_feature_matrix(df)

    best = {'k': None, 'sil': -1, 'labels': None}
    for k in K_CANDIDATES:
        km = KMeans(n_clusters=k, n_init='auto', random_state=RANDOM_STATE)
        labels = km.fit_predict(X)
        uniq = np.unique(labels)
        if len(uniq) < 2 or len(uniq) == len(labels):
            continue
        sil = silhouette_score(X, labels)
        if sil > best['sil']:
            best = {'k': k, 'sil': sil, 'labels': labels}

    if best['k'] is None:
        print(f"[WARN] {name}: uygun k bulunamadı.")
        km_summary.append({'name': name, 'k': None, 'silhouette': None})
        continue

    out_df = df[['title']].copy(); out_df['cluster'] = best['labels']
    out_csv = OUT_DIR / f"{name}_kmeans_k{best['k']}.csv"
    out_df.to_csv(out_csv, index=False)

    out_png = OUT_DIR / f"{name}_kmeans_pca.png"
    pca_plot(X, best['labels'], f"{name} K-Means (k={best['k']}, sil={best['sil']:.2f})", out_png)

    print(f"[OK][K-Means] {name}: k={best['k']} sil={best['sil']:.3f} -> {out_csv.name}")
    preview_clusters(df['title'], best['labels'])
    km_summary.append({'name': name, 'k': best['k'], 'silhouette': round(best['sil'],3)})

pd.DataFrame(km_summary).to_csv(OUT_DIR / 'kmeans_summary.csv', index=False)
print('\nK-Means özet ->', OUT_DIR / 'kmeans_summary.csv')


# 15) DBSCAN (Per-Type)
EPS ve min_samples grid'i üzerinde süpürme yapar. Performansı kümelerin sayısı, gürültü oranı (label = -1) ve varsa silhouette (≥2 küme) ile raporlar. En iyi konfigürasyonu seçip kaydeder.

In [ ]:
db_summary = []
for name, path in SIGNATURE_FILES:
    p = Path(path)
    if not p.exists():
        print(f"[SKIP] {name}: {path} bulunamadı.")
        continue

    df = load_signatures(str(p))
    X, sig_cols = get_feature_matrix(df)

    best = {'eps': None, 'min': None, 'score': -np.inf, 'labels': None, 'k': None, 'noise': None}
    # Basit skor: (küme sayısı >=2) ve (noise oranı düşük) ve (silhouette yüksekse avantaj)
    for eps in DBSCAN_EPS_GRID:
        for m in DBSCAN_MIN_SAMPLES:
            db = DBSCAN(eps=eps, min_samples=m)
            labels = db.fit_predict(X)
            uniq = np.unique(labels)
            k = len(uniq[uniq!=-1])  # noise hariç cluster sayısı
            noise_ratio = (labels==-1).mean()
            # Silhouette yalnızca 2+ küme ve tüm tekil olmayan durumda anlamlıdır
            sil = None
            try:
                if k >= 2 and (labels!=-1).sum() > 2:
                    sil = silhouette_score(X[labels!=-1], labels[labels!=-1])
            except Exception:
                sil = None
            # basit bir skor fonksiyonu: küme sayısı ve silhouette lehine, noise aleyhine
            score = (k * 0.5) + ((sil or 0) * 1.0) - (noise_ratio * 0.3)
            if score > best['score']:
                best = {'eps': eps, 'min': m, 'score': score, 'labels': labels,
                        'k': k, 'noise': round(noise_ratio,3)}

    labels = best['labels']
    out_df = df[['title']].copy(); out_df['cluster'] = labels
    out_csv = OUT_DIR / f"{name}_dbscan_eps{best['eps']}_min{best['min']}.csv"
    out_df.to_csv(out_csv, index=False)

    # PCA görsel
    out_png = OUT_DIR / f"{name}_dbscan_pca.png"
    pca_plot(X, labels, f"{name} DBSCAN (eps={best['eps']}, min={best['min']}, k={best['k']}, noise={best['noise']})", out_png)

    print(f"[OK][DBSCAN] {name}: eps={best['eps']} min={best['min']} k={best['k']} noise={best['noise']} -> {out_csv.name}")
    # Gürültü (-1) harici örnekler
    if (labels!=-1).any():
        preview_clusters(df['title'][labels!=-1], labels[labels!=-1])
    db_summary.append({'name': name, 'eps': best['eps'], 'min_samples': best['min'], 'clusters': best['k'], 'noise_ratio': best['noise'], 'score': round(best['score'],3)})

pd.DataFrame(db_summary).to_csv(OUT_DIR / 'dbscan_summary.csv', index=False)
print('\nDBSCAN özet ->', OUT_DIR / 'dbscan_summary.csv')


# 16) Combined (Multi-View) Clustering — K-Means ve DBSCAN
Birden çok imza tablosunu `title` üzerinde birleştirir; ardından K-Means ve DBSCAN uygular.

In [ ]:
from functools import reduce
loaded = []
for name, path in SIGNATURE_FILES:
    p = Path(path)
    if not p.exists():
        continue
    df = load_signatures(str(p))
    sig_cols = [c for c in df.columns if c.startswith('s')]
    df = df.rename(columns={c: f"{name}_{c}" for c in sig_cols})
    loaded.append(df)

if len(loaded) >= 2:
    merged = reduce(lambda l, r: pd.merge(l, r, on='title', how='inner'), loaded)
    feat_cols = [c for c in merged.columns if c != 'title']
    Xc = merged[feat_cols].values
    if SCALE_FEATURES:
        Xc = StandardScaler(with_mean=False).fit_transform(Xc)

    # K-Means (best k)
    best = {'k': None, 'sil': -1, 'labels': None}
    for k in K_CANDIDATES:
        km = KMeans(n_clusters=k, n_init='auto', random_state=RANDOM_STATE)
        labels = km.fit_predict(Xc)
        uniq = np.unique(labels)
        if len(uniq) < 2 or len(uniq) == len(labels):
            continue
        sil = silhouette_score(Xc, labels)
        if sil > best['sil']:
            best = {'k': k, 'sil': sil, 'labels': labels}

    if best['k'] is not None:
        out_df = merged[['title']].copy(); out_df['cluster'] = best['labels']
        out_csv = OUT_DIR / f"combined_kmeans_k{best['k']}.csv"
        out_df.to_csv(out_csv, index=False)
        out_png = OUT_DIR / f"combined_kmeans_pca.png"
        pca_plot(Xc, best['labels'], f"combined K-Means (k={best['k']}, sil={best['sil']:.2f})", out_png)
        print(f"[OK][combined K-Means] k={best['k']} sil={best['sil']:.3f} -> {out_csv.name}")

    # DBSCAN combined (grid sweep)
    best_db = {'eps': None, 'min': None, 'score': -np.inf, 'labels': None, 'k': None, 'noise': None}
    for eps in DBSCAN_EPS_GRID:
        for m in DBSCAN_MIN_SAMPLES:
            db = DBSCAN(eps=eps, min_samples=m)
            labels = db.fit_predict(Xc)
            uniq = np.unique(labels)
            k = len(uniq[uniq!=-1])
            noise_ratio = (labels==-1).mean()
            sil = None
            try:
                if k >= 2 and (labels!=-1).sum() > 2:
                    sil = silhouette_score(Xc[labels!=-1], labels[labels!=-1])
            except Exception:
                sil = None
            score = (k * 0.5) + ((sil or 0) * 1.0) - (noise_ratio * 0.3)
            if score > best_db['score']:
                best_db = {'eps': eps, 'min': m, 'score': score, 'labels': labels,
                           'k': k, 'noise': round(noise_ratio,3)}

    labels = best_db['labels']
    out_df = merged[['title']].copy(); out_df['cluster'] = labels
    out_csv = OUT_DIR / f"combined_dbscan_eps{best_db['eps']}_min{best_db['min']}.csv"
    out_df.to_csv(out_csv, index=False)
    out_png = OUT_DIR / f"combined_dbscan_pca.png"
    pca_plot(Xc, labels, f"combined DBSCAN (eps={best_db['eps']}, min={best_db['min']}, k={best_db['k']}, noise={best_db['noise']})", out_png)
    print(f"[OK][combined DBSCAN] eps={best_db['eps']} min={best_db['min']} k={best_db['k']} noise={best_db['noise']} -> {out_csv.name}")
else:
    print('[INFO] combined: en az iki imza dosyası gerekli (şu an <2).')


# 17) Raporlama & Sonraki Adımlar
- `kmeans_summary.csv` ve `dbscan_summary.csv` dosyalarını raporda tablo olarak kullanın.
- PCA görsellerinden örnekleri ekleyin.
- Eğer ekip ek imza setleri (17 parçayı) push ederse `SIGNATURE_FILES` listesine ekleyip bu notebook'u tekrar çalıştırın.
- (Opsiyonel) DBSCAN için k-distance grafiği, K-Means için Davies–Bouldin skoru ekleyebilirsiniz.

In [ ]:
# Özetleri göster
km_sum = OUT_DIR / 'kmeans_summary.csv'
db_sum = OUT_DIR / 'dbscan_summary.csv'
if km_sum.exists():
    print('K-Means Summary:')
    display(pd.read_csv(km_sum))
else:
    print('[INFO] kmeans_summary.csv yok')
if db_sum.exists():
    print('\nDBSCAN Summary:')
    display(pd.read_csv(db_sum))
else:
    print('[INFO] dbscan_summary.csv yok')
